# Appendix 03 - Data preprocessing: *CMIP6 data*

In [10]:
import xarray as xr
import os
import numpy as np
import cftime 
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import mapping
import dask 

# dask.config.set(**{'array.slicing.split_large_chunks': True})
import pandas as pd

In [11]:
world_boundary_file     = '/home/jgrassi/work/XSeasonsDetect/INDIA/data/raw/shapefiles/boundary.gpkg' 
world_boundary      = gpd.read_file(world_boundary_file)
world_boundary

,GID_0,COUNTRY,geometry
0,IDN,Indonesia,"MULTIPOLYGON (((95.39403 4.90093, 95.39281 4.9..."
1,MYS,Malaysia,"MULTIPOLYGON (((100.03721 6.06027, 100.03757 6..."
2,LKA,Sri Lanka,"MULTIPOLYGON (((80.93069 6.07597, 80.93069 6.0..."
3,MDV,Maldives,"MULTIPOLYGON (((73.07653 5.11486, 73.07708 5.1..."
4,THA,Thailand,"MULTIPOLYGON (((100.09992 6.49741, 100.09963 6..."
5,LAO,Laos,"MULTIPOLYGON (((100.09941 20.25902, 100.09897 ..."
6,MMR,Myanmar,"MULTIPOLYGON (((97.80139 8.82583, 97.80084 8.8..."
7,IND,India,"MULTIPOLYGON (((76.97486 8.38514, 76.97486 8.3..."
8,PAK,Pakistan,"MULTIPOLYGON (((68.09181 23.70597, 68.09097 23..."
9,Z09,India,"MULTIPOLYGON (((78.91309 31.26072, 78.91409 31..."


Path for complete file:
- Temperature:      `../data/raw/ERA5/2m_temperature`
- Precipitation:    `../data/raw/ERA5/total_precipitation`

## STEP 1: regridding

- Define a target grid:


gridtype = lonlat   <br />
xsize    = 360      <br />
ysize    = 180      <br />
xfirst   = −179.5   <br />
xinc     = 1        <br />
yfirst   = -89.5    <br />
yinc     = 1        <br />


In [12]:
def sel_years(dataset, start, end):

    dataset = dataset.sel(time=slice(start, end)).chunk(dict(time=-1))
    return dataset


def interpolate_na(dataset):

    try:
        ### FILLING NA IN DATASET TP
        # Verifica l'asse temporale del dataset
        time = dataset.time

        # Crea un indice completo con frequenza oraria
        complete_time_index = pd.date_range(start=time.min().item(), end=time.max().item(), freq='D')

    except:
        dataset = dataset.convert_calendar('standard')
        ### FILLING NA IN DATASET TP
        # Verifica l'asse temporale del dataset
        time = dataset.time

        # Crea un indice completo con frequenza oraria
        complete_time_index = pd.date_range(start=time.min().item(), end=time.max().item(), freq='D')
        
    # Reindicizza il dataset per includere tutte le date, anche quelle mancanti
    ds_reindexed = dataset.reindex(time=complete_time_index)

    # Interpola i dati per riempire i valori mancanti
    dataset = dataset = ds_reindexed.interpolate_na(dim='time', method='linear')

    return dataset



def clean_cut(dataset, boundary = None, window = None, remove_empty = True):

    # Converting calendar and removing useless dimensions
    dataset = dataset.convert_calendar('noleap')
    dataset = dataset.drop_dims('bnds')

    if boundary is not None:
        # Setting the datasets for masking
        dataset.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        dataset.rio.write_crs("epsg:4326", inplace=True)

        # Masking the datasets
        dataset = dataset.rio.clip(boundary.geometry.apply(mapping), boundary.crs, drop=True)

    if window is not None:
        dataset = dataset.rolling(time=window, center=True).mean()

    
    if remove_empty:

        start_year = dataset.time.dt.year.min().values
        end_year = dataset.time.dt.year.max().values
        dataset = dataset.sel(time=slice(str(start_year+1),str(end_year-1)))

    return dataset


def remap_cdo(in_file, out_file, grid_file):

    os.system(f'cdo remapbil,{grid_file} {in_file} {out_file}')




def standard_preprocess(in_path, temp_path, out_path, start_year, end_year, grid_file, boundary = None, window = None, remove_empty = True, out_filename = 'final'):

    remap_cdo(f'{in_path}/*.nc', f'{temp_path}/temp.nc', grid_file)

    raw_dataset = xr.open_mfdataset(f'{temp_path}/temp.nc')

    dataset_work = sel_years(raw_dataset, str(start_year),str(end_year))
    dataset_work = interpolate_na(dataset_work)
    dataset_work = clean_cut(dataset_work, boundary, window, remove_empty)

    dataset_work.to_netcdf(f'{out_path}/{out_filename}.nc')

    os.remove(f'{temp_path}/temp.nc')

In [13]:
# -- MAKING FOLDER STRUCTURE

target_grid_path    = '../data/preprocessed/ERA5/target_grid.txt' 

base_paths = ['../data/raw/CMIP6', '../data/temp/CMIP6', '../data/preprocessed/CMIP6']

models = [
    'EC-Earth3'
    ## 'AWI-CM-1-1-MR',
    # 'MIROC6',
    # 'INM-CM5-0',
    #'CMCC-ESM2',
    #'ACCESS-ESM1-5',
    ## 'MPI-ESM1-2-HR'
      ]


experiments = ['ssp585']
variables = ['pr']

repository_paths = [
                    "/data/datasets/synda/data/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3/ssp585/r1i1p1f1/day/pr/gr/v20200310/"
                    #'/work/datasets/synda/data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/historical/r1i1p1f1/day/tas/gr/v20200310/',
                    ##'/work/datasets/synda/data/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/{experiment}/r1i1p1f1/day/{variable}/gn/v20190529/',
                    #'/work/datasets/synda/data/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/day/tas/gn/v20191016/',
                    #'/work/datasets/synda/data/CMIP6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/day/tas/gr1/v20190610/',
                    #'/work/datasets/synda/data/CMIP6/CMIP/CMCC/CMCC-ESM2/historical/r1i1p1f1/day/tas/gn/v20210114/',
                    #'/work/datasets/synda/data/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical/r1i1p1f1/day/tas/gn/v20191115/',
                    ##'/work/datasets/synda/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/{experiment}/r1i1p1f1/day/{variable}/gn/v20190710/'
                    ]

for model, repository_path in zip(models, repository_paths):
    for experiment in experiments:
        for variable in variables:

            for path in base_paths:

                try:
                    os.makedirs(f'{path}/{model}/{experiment}/{variable}')
                except:
                    pass
            
                if path == '../data/raw/CMIP6':

                    try:
                        repository_path_n = repository_path
                        os.system(f'cdo sellonlatbox,65,100,5,40 -mergetime {repository_path_n}*.nc {path}/{model}/{experiment}/{variable}/{model}-raw-{variable}.nc')
                    except:
                        raise
                
            raw_path, temp_path, preprocess_path = [f'{bp}/{model}/{experiment}/{variable}' for bp in base_paths]

            standard_preprocess(raw_path, temp_path, preprocess_path, 2020, 2100, target_grid_path, world_boundary, 15, True, f'{model}-{experiment}-{variable}-{2020}-{2100}')

cdo(1) mergetime: Process started
cdo(1) mergetime: Processed 4117102592 values from 86 variables over 31411 timesteps.
cdo    sellonlatbox: Processed 4117102592 values from 1 variable over 31411 timesteps [105.68s 221MB].
cdo    remapbil: Bilinear weights from gaussian (50x50) to lonlat (41x36) grid
cdo    remapbil: Processed 78527500 values from 1 variable over 31411 timesteps [3.78s 132MB].


In [14]:
# # -- MAKING FOLDER STRUCTURE

# target_grid_path    = '../data/preprocessed/ERA5/target_grid.txt' 

# base_paths = ['../data/raw/CMIP6', '../data/temp/CMIP6', '../data/preprocessed/CMIP6']

# models = [
#     #'EC-Earth3',
#     #'AWI-CM-1-1-MR',
#     #'MIROC6',
#     #'INM-CM5-0',
#     #'CMCC-ESM2',
#     #'ACCESS-ESM1-5',
#     #'MPI-ESM1-2-HR'
#       ]


# experiments = ['ssp585']
# variables = ['pr', 'tas']

# repository_paths = [
#                     #'/work/datasets/synda/data/CMIP6/ScenarioMIP/EC-Earth-Consortium/EC-Earth3/{experiment}/r1i1p1f1/day/{variable}/gr/v20200310/',
#                     #'/work/datasets/synda/data/CMIP6/ScenarioMIP/AWI/AWI-CM-1-1-MR/{experiment}/r1i1p1f1/day/{variable}/gn/v20190529/',
#                     #'/work/datasets/synda/data/CMIP6/ScenarioMIP/MIROC/MIROC6/{experiment}/r1i1p1f1/day/{variable}/gn/v20191016/',
#                     #'/work/datasets/synda/data/CMIP6/ScenarioMIP/INM/INM-CM5-0/{experiment}/r1i1p1f1/day/{variable}/gr1/v20190724/',
#                     #'/work/datasets/synda/data/CMIP6/ScenarioMIP/CMCC/CMCC-ESM2/{experiment}/r1i1p1f1/day/{variable}/gn/v20210126/',
#                     '/work/datasets/synda/data/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/{experiment}/r1i1p1f1/day/{variable}/gn/v20191115/',
#                     '/work/datasets/synda/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/{experiment}/r1i1p1f1/day/{variable}/gn/v20190710/'
#                     ]

# for model, repository_path in zip(models, repository_paths):
#     for experiment in experiments:
#         for variable in variables:

            

#             for path in base_paths:

#                 try:
#                     os.makedirs(f'{path}/{model}/{experiment}/{variable}')
#                 except:
#                     pass
            
#                 if path == '../data/raw/CMIP6':

#                     try:
#                         repository_path_n = repository_path.format(model=model, experiment=experiment, variable=variable)
#                         os.system(f'cdo sellonlatbox,65,100,5,40 -mergetime {repository_path_n}*.nc {path}/{model}/{experiment}/{variable}/{model}-raw-{variable}.nc')
#                     except:
#                         raise
                
#             raw_path, temp_path, preprocess_path = [f'{bp}/{model}/{experiment}/{variable}' for bp in base_paths]

#             standard_preprocess(raw_path, temp_path, preprocess_path, 2019, 2041, target_grid_path, world_boundary, 15, True, 'short_term_2020-2040')
#             standard_preprocess(raw_path, temp_path, preprocess_path, 2038, 2071, target_grid_path, world_boundary, 15, True, 'medium_term_2040-2070')
#             standard_preprocess(raw_path, temp_path, preprocess_path, 2069, 2199, target_grid_path, world_boundary, 15, True, 'long_term_2070-2100')

### ERA5

In [15]:
# raw_path, temp_path, preprocess_path = [{bp}/{model}/{experiment}/{variable} for bp in base_paths]

In [16]:
# raw_data_paths       = ['../data/raw/ERA5/2m_temperature', '../data/raw/ERA5/total_precipitation']
# temp_data_paths      = ['../data/temp/ERA5/2m_temperature', '../data/temp/ERA5/total_precipitation']
# preproc_data_paths   = ['../data/preprocessed/ERA5/2m_temperature', '../data/preprocessed/ERA5/total_precipitation']

# target_grid_path    = '../data/preprocessed/ERA5/target_grid.txt' 


In [17]:
# for raw_path, temp_path, preprocess_path in zip(raw_data_paths,temp_data_paths,preproc_data_paths):

#     standard_preprocess(raw_path, temp_path, preprocess_path, 1968, 2019, target_grid_path, world_boundary, 15, True, 'final')

In [18]:
# # raw_data_paths       = ['../data/raw/CMIP6/EC-Earth3/ssp585/2m_temperature', '../data/raw/CMIP6/EC-Earth3/ssp585/total_precipitation']
# # temp_data_paths      = ['../data/temp/CMIP6/EC-Earth3/ssp585/2m_temperature', '../data/temp/CMIP6/EC-Earth3/ssp585/total_precipitation']
# # preproc_data_paths   = ['../data/preprocessed/CMIP6/EC-Earth3/ssp585/2m_temperature', '../data/preprocessed/CMIP6/EC-Earth3/ssp585/total_precipitation']

# raw_data_paths       = ['../data/raw/CMIP6/EC-Earth3/ssp585/total_precipitation']
# temp_data_paths      = ['../data/temp/CMIP6/EC-Earth3/ssp585/total_precipitation']
# preproc_data_paths   = ['../data/preprocessed/CMIP6/EC-Earth3/ssp585/total_precipitation']

# target_grid_path    = '../data/preprocessed/ERA5/target_grid.txt' 

In [19]:
# for raw_path, temp_path, preprocess_path in zip(raw_data_paths,temp_data_paths,preproc_data_paths):

#     standard_preprocess(raw_path, temp_path, preprocess_path, 2019, 2041, target_grid_path, world_boundary, 15, True, 'short_term_2020-2040')
#     standard_preprocess(raw_path, temp_path, preprocess_path, 2038, 2071, target_grid_path, world_boundary, 15, True, 'medium_term_2040-2070')
#     standard_preprocess(raw_path, temp_path, preprocess_path, 2069, 2199, target_grid_path, world_boundary, 15, True, 'long_term_2070-2100')